# Generar datos para la distribución de servicios de justicia

## Importar librerías

In [1]:
import pandas as pd
import numpy as np
import os
import re

## Leer los datos

In [ ]:
path = f"./dataset/variables_alternas/Distribución_Territorial_de_las_instalaciones_de_servicios_de_justicia.xlsx"
sheet_names = pd.ExcelFile(path).sheet_names
sheet_names

['1.1 OJ',
 '1.2 MP ',
 '1.3 IDDP ',
 '1.4 PNC ',
 'Visualización de datos OJ',
 'Visualización de datos MP',
 'Visualización de datos IDPP',
 'Visualización de datos PNC']

In [3]:
dfs = [pd.read_excel(path, sheet_name=sheet, dtype=str) for sheet in sheet_names][:4]
for index, df in enumerate(dfs):
    print(df.info())
    dfs[index] = df.dropna(how="all")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 12 columns):
 #   Column                                                                             Non-Null Count  Dtype 
---  ------                                                                             --------------  ----- 
 0   Ubicación                                                                          365 non-null    object
 1   Salas (mixtas)                                                                     359 non-null    object
 2   Tribunales de Sentencia                                                            357 non-null    object
 3   Juzgados de Primera Instancia Penal                                                357 non-null    object
 4   Juzgados de Primera Instancia Penal, Narcoactividad y Delitos contral el Ambiente  338 non-null    object
 5   Juzgados de Adolescentes y Niñez                                                   357 non-null    object
 6   Ju

In [4]:
oj = dfs[0] #organismo_judicial
mp = dfs[1] #ministerio_publico
iddp = dfs[2] #instituto_defensa_publica_penal
pnc = dfs[3] #pnc

## Procesar las columnas del dataframe

In [5]:
# Se bifurca en zacapa
def process_dataframe(df):
    df = df.iloc[:df[df['Ubicación ']=="Zacapa"].index[1]+1]
    return df
oj = process_dataframe(oj).replace('-', np.nan)
mp = process_dataframe(mp).replace('-', np.nan)
iddp = process_dataframe(iddp).replace('-', np.nan)
pnc = process_dataframe(pnc).replace('-', np.nan)

In [6]:
# Renombrar las columnas
mp = mp.rename(columns=mp.iloc[0])
mp = mp.iloc[1:]
mp = mp.rename(columns={np.nan:"Ubicación "})
mp.reset_index(drop=True, inplace=True)

## Extraer los departamentos de las ubicaciones

In [7]:
departamentos = ["Guatemala", "El Progreso", "Sacatepéquez", "Chimaltenango", "Escuintla", "Santa Rosa", "Jutiapa", "Zacapa", "Chiquimula", "Jalapa", "Totonicapán", "Quetzaltenango", "Suchitepéquez", "Retalhuleu", "San Marcos", "Huehuetenango", "Quiché", "Baja Verapaz", "Alta Verapaz", "Petén", "Izabal", "Sololá"]
len(departamentos)

22

In [8]:
indexes = {dep: i for i, dep in enumerate(departamentos)}
for departamento in departamentos:
    indexes[departamento] = (
        oj[oj["Ubicación "] == departamento].index[0], 
        mp[mp["Ubicación "] == departamento].index[0], 
        iddp[iddp["Ubicación "] == departamento].index[0], 
        pnc[pnc["Ubicación "] == departamento].index[0]
    )

In [9]:
indexes.items()

dict_items([('Guatemala', (np.int64(80), np.int64(80), np.int64(80), np.int64(80))), ('El Progreso', (np.int64(56), np.int64(56), np.int64(56), np.int64(56))), ('Sacatepéquez', (np.int64(236), np.int64(236), np.int64(236), np.int64(238))), ('Chimaltenango', (np.int64(27), np.int64(27), np.int64(27), np.int64(27))), ('Escuintla', (np.int64(65), np.int64(65), np.int64(65), np.int64(65))), ('Santa Rosa', (np.int64(284), np.int64(284), np.int64(283), np.int64(286))), ('Jutiapa', (np.int64(146), np.int64(146), np.int64(146), np.int64(147))), ('Zacapa', (np.int64(350), np.int64(350), np.int64(349), np.int64(352))), ('Chiquimula', (np.int64(44), np.int64(44), np.int64(44), np.int64(44))), ('Jalapa', (np.int64(138), np.int64(138), np.int64(138), np.int64(139))), ('Totonicapán', (np.int64(341), np.int64(341), np.int64(340), np.int64(343))), ('Quetzaltenango', (np.int64(179), np.int64(179), np.int64(179), np.int64(181))), ('Suchitepéquez', (np.int64(319), np.int64(319), np.int64(318), np.int64(3

In [10]:
# Extraer los departamentos
oj_departamentos = pd.concat([oj.iloc[tupla[0]] for _, tupla in indexes.items()], axis=1).T.sort_index()
mp_departamentos = pd.concat([mp.iloc[tupla[1]] for _, tupla in indexes.items()], axis=1).T.sort_index()
iddp_departamentos = pd.concat([iddp.iloc[tupla[2]] for _, tupla in indexes.items()], axis=1).T.sort_index()
pnc_departamentos = pd.concat([pnc.iloc[tupla[3]] for _, tupla in indexes.items()], axis=1).T.sort_index()
# Darle el departamento de esas ubicaciones
for df in [oj_departamentos, mp_departamentos, iddp_departamentos, pnc_departamentos]:
    df["depto"] = df["Ubicación "].copy()

In [11]:
# Se rellenan los valores cuando se encuentre uno nuevo en departamento
def departamento_columns(original_df: pd.DataFrame, df_departamentos: pd.DataFrame) -> pd.DataFrame:
    temp_df = original_df.merge(df_departamentos, on='Ubicación ', how="left", suffixes=('', '_dept'))
    print(temp_df.info())
    for col in df_departamentos.columns[1:]:
        if col != "depto":
            temp_df[col+'_dept'] = temp_df[col+'_dept'].fillna(method='ffill')
        else:
            temp_df[col] = temp_df[col].fillna(method='ffill')
    return temp_df

In [12]:
# Reset index to ensure unique, sequential indices for slicing
oj_final = departamento_columns(oj, oj_departamentos)
mp_final = departamento_columns(mp, mp_departamentos)
iddp_final = departamento_columns(iddp, iddp_departamentos)
pnc_final = departamento_columns(pnc, pnc_departamentos)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363 entries, 0 to 362
Data columns (total 24 columns):
 #   Column                                                                                  Non-Null Count  Dtype 
---  ------                                                                                  --------------  ----- 
 0   Ubicación                                                                               363 non-null    object
 1   Salas (mixtas)                                                                          356 non-null    object
 2   Tribunales de Sentencia                                                                 356 non-null    object
 3   Juzgados de Primera Instancia Penal                                                     356 non-null    object
 4   Juzgados de Primera Instancia Penal, Narcoactividad y Delitos contral el Ambiente       337 non-null    object
 5   Juzgados de Adolescentes y Niñez                                              

/tmp/ipykernel_133427/1012609957.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  temp_df[col+'_dept'] = temp_df[col+'_dept'].fillna(method='ffill')
/tmp/ipykernel_133427/1012609957.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  temp_df[col] = temp_df[col].fillna(method='ffill')
/tmp/ipykernel_133427/1012609957.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  temp_df[col+'_dept'] = temp_df[col+'_dept'].fillna(method='ffill')
/tmp/ipykernel_133427/1012609957.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  temp_df[col] = temp_df[col].fillna(method='ffill')
/tmp/ipykernel_133427/1012609957.py:7: FutureWarning: Series.fillna 

## Eliminar aquellos departamentos que no sean departamentos

In [13]:
def eliminate_depto_rows(original_df: pd.DataFrame, depto_list:list) -> pd.DataFrame:
    return original_df[~original_df["Ubicación "].isin(depto_list)]

In [14]:
oj_final = eliminate_depto_rows(oj_final, departamentos)
mp_final = eliminate_depto_rows(mp_final, departamentos)
iddp_final = eliminate_depto_rows(iddp_final, departamentos)
pnc_final = eliminate_depto_rows(pnc_final, departamentos)

## Unificar todos los valores de oj, mp, iddp, y pnc

In [ ]:
df_final = oj_final\
    .merge(mp_final, how="outer", on="Ubicación ", suffixes=("_oj", "_mp"))\
    .merge(iddp_final, how="outer", on="Ubicación ", suffixes=("", "_iddp"))\
    .merge(pnc_final, how="outer", on="Ubicación ", suffixes=("", "_pnc"))\
    .fillna(0)
    # .to_csv(
    #     f"./dataset/variables_alternas_procesadas/distribucion_de_servicios_de_justicia.csv", 
    #     index=False
    # )

/tmp/ipykernel_133427/3874541042.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(0)


## Mapear los municipios a los diferentes departamentos

In [16]:

municipios_departamentos = [
    ("San José el Golfo", "Guatemala"),
    ("San Raimundo", "Guatemala"),
    ("Petapa", "Guatemala"),
    ("Guatemala", "Guatemala"),
    ("Patulul", "Suchitepéquez"),
    ("Alotenango", "Sacatepéquez"),
    ("Comalapa", "Chimaltenango"),
    ("Yepocapa", "Chimaltenango"),
    ("San Rafael las Flores", "Santa Rosa"),
    ("San Lucas Toliman", "Sololá"),
    ("San Cristobal Totonicapán", "Totonicapán"),
    ("Santa Lucía La Reforma", "Totonicapán"),
    ("Ostuncalco", "Quetzaltenango"),
    ("Olintepeque", "Quetzaltenango"),
    ("Colomba", "Quetzaltenango"),
    ("Palestina de los Altos", "Quetzaltenango"),
    ("San José El Idolo", "Suchitepéquez"),
    ("San Pablo Jocopilas", "Suchitepéquez"),
    ("Sololá", "Sololá"),
    ("Samayac", "Suchitepéquez"),
    ("Rio Bravo", "Escuintla"),
    ("Mixco", "Guatemala"),
    ("San Rafael Pié de La Cuesta", "San Marcos"),
    ("El Rodeo", "San Marcos"),
    ("Ocós", "San Marcos"),
    ("Ixtahuacán", "Huehuetenango"),
    ("Unión Cantinil", "Huehuetenango"),
    ("Chichicastenango", "Quiché"),
    ("Uspantán", "Quiché"),
    ("Ixcán", "Quiché"),
    ("Momostenango", "Totonicapán"),
    ("Nebaj", "Quiché"),
    ("Tucurú", "Alta Verapaz"),
    ("Lanquín", "Alta Verapaz"),
    ("Cahabón", "Alta Verapaz"),
    ("Santa Catalina la Tinta", "Alta Verapaz"),
    ("Jocotán", "Chiquimula"),
    ("Quetzaltepeque", "Chiquimula"),
    ("Villa Nueva", "Guatemala"),
    ("Pochuta", "Chimaltenango"),
    ("Cuilapa", "Santa Rosa"),
    ("San Bartolo", "Totonicapán"),
    ("Santo Tomás La Unión", "Suchitepéquez"),
    ("Rio Blanco", "San Marcos"),
    ("Barillas", "Huehuetenango"),
    ("Chinique", "Quiché"),
    ("El Chol", "Baja Verapaz"),
    ("San Juan Erminta", "Chiquimula"),
    ("San Jacinto", "Chiquimula"),
    ("San José Pinula", "Guatemala"),
    ("Amatitlán", "Guatemala"),
    ("Amatitlan", "Guatemala"),
    ("Patzicía", "Chimaltenango"),
    ("San Juan Cotzal", "Quiché"),
    ("Sacapulas", "Quiché"),
    ("Chajul", "Quiché"),
    ("Melchor de Mencos", "Petén"),
    ("La Unión", "Zacapa"),
    ("Soloma", "Huehuetenango"),
    ("Cunén", "Quiché"),
    ("San José Acatempa", "Jutiapa"),
    ("Antigua Guatemala", "Sacatepéquez"),
    ("Chimaltenango", "Chimaltenango"),
    ("Cabricán", "Quetzaltenango"),
    ("San Mateo Ixtatán", "Huehuetenango"),
    ("Río Hondo", "Zacapa"),
    ("Jerez", "Jutiapa"),
    ("San Miguel Dueñas", "Sacatepéquez"),
    ("Santa Cruz Balanyá", "Chimaltenango"),
    ("Escuintla", "Escuintla"),
    ("Quetzaltenango", "Quetzaltenango"),
    ("Nueva Concepción", "Escuintla"),
    ("Nentón", "Huehuetenango"),
    ("Panzós", "Alta Verapaz"),
    ("Santa Cruz del Quiché", "Quiché"),
    ("San Francisco El Alto", "Totonicapán"),
    ("San Juan Sacatepéquez", "Guatemala"),
    ("Chiquimula", "Chiquimula"),
    ("Nuevo Progreso", "San Marcos"),
    ("Huehuetenango", "Huehuetenango"),
    ("Chiché", "Quiché"),
    ("San Juan Ixcoy", "Huehuetenango"),
    ("Santa Eulalia", "Huehuetenango"),
    ("San Antonio Palopó", "Sololá"),
    ("Senahú", "Alta Verapaz"),
    ("Totonicapán", "Totonicapán"),
    ("La Esperanza", "Quetzaltenango"),
    ("Mazatenango", "Suchitepéquez"),
    ("San Rafael La Independencia", "Huehuetenango"),
    ("Chicamán", "Quiché"),
    ("El Quetzal", "San Marcos"),
    ("San Martín Sacatepéquez", "Quetzaltenango"),
    ("Ignorado", "Desconocido"),
    ("El Tumbador", "San Marcos"),
    ("Santo Domingo Suchitepéquez", "Suchitepéquez"),
    ("Chahal", "Alta Verapaz"),
    ("Zacualpa", "Quiché"),
    ("Fray Bartolomé de Las Casas", "Alta Verapaz"),
    ("Oratorio", "Santa Rosa"),
    ("Villa Canales", "Guatemala"),
    ("Retalhuleu", "Retalhuleu"),
    ("Almolonga", "Quetzaltenango"),
    ("Guastatoya", "El Progreso"),
    ("Chuarrancho", "Guatemala"),
    ("Atescatempa", "Jutiapa"),
    ("Chinautla", "Guatemala"),
    ("San Juan Chamelco", "Alta Verapaz"),
    ("Tamahú", "Alta Verapaz"),
    ("San Sebastián Huehuetenango", "Huehuetenango"),
    ("Cajolá", "Quetzaltenango"),
    ("Flores", "Petén"),
    ("San Pablo", "San Marcos"),
    ("Tecpán Guatemala", "Chimaltenango"),
    ("Iztapa", "Escuintla"),
    ("La Libertad", "Petén"),
    ("Tiquisate", "Escuintla"),
    ("Esquipulas", "Chiquimula"),
    ("Pachalum", "Quiché"),
    ("Malacatán", "San Marcos"),
    ("San Lorenzo", "San Marcos"),
    ("San Miguel Siguilá", "Quetzaltenango"),
    ("Nahualá", "Sololá"),
    ("Santa María de Jesús", "Sacatepéquez"),
    ("Camotán", "Chiquimula"),
    ("Olopa", "Chiquimula"),
    ("San Vicente Pacaya", "Escuintla"),
    ("San José La Arada", "Chiquimula"),
    ("Pajapita", "San Marcos"),
    ("San Pedro Jocopilas", "Quiché"),
    ("Fraijanes", "Guatemala"),
    ("Palencia", "Guatemala"),
    ("Santa Catarina Pinula", "Guatemala"),
    ("Magdalena Milpas Altas", "Sacatepéquez"),
    ("Siquinalá", "Escuintla"),
    ("Santa Clara La Laguna", "Sololá"),
    ("El Tejar", "Chimaltenango"),
    ("San José Poaquil", "Chimaltenango"),
    ("Sibilia", "Quetzaltenango"),
    ("Casillas", "Santa Rosa"),
    ("Panajachel", "Sololá"),
    ("San Andrés Itzapa", "Chimaltenango"),
    ("Salcajá", "Quetzaltenango"),
    ("Coatepeque", "Quetzaltenango"),
    ("Taxisco", "Santa Rosa"),
    ("El Palmar", "Quetzaltenango"),
    ("San Francisco Zapotitlán", "Suchitepéquez"),
    ("Chiquimulilla", "Santa Rosa"),
    ("Jalpatagua", "Jutiapa"),
    ("San Bernardino", "Suchitepéquez"),
    ("Santa Catarina Ixtahuacán", "Sololá"),
    ("Santa Lucía Cotzumalguapa", "Escuintla"),
    ("San Pedro Sacatepéquez", "Guatemala"),
    ("San Marcos", "San Marcos"),
    ("Todos Santos Cuchumatán", "Huehuetenango"),
    ("San Pedro Necta", "Huehuetenango"),
    ("Guanagazapa", "Escuintla"),
    ("Colotenango", "Huehuetenango"),
    ("Canillá", "Quiché"),
    ("Sipacapa", "San Marcos"),
    ("Santa Ana Huista", "Huehuetenango"),
    ("Joyabaj", "Quiché"),
    ("San Antonio Ilotenango", "Quiché"),
    ("Sibinal", "San Marcos"),
    ("San Andrés Sajcabajá", "Quiché"),
    ("San Pedro Carchá", "Alta Verapaz"),
    ("Tactic", "Alta Verapaz"),
    ("Cobán", "Alta Verapaz"),
    ("San Cristóbal Verapaz", "Alta Verapaz"),
    ("Chisec", "Alta Verapaz"),
    ("San Antonio Suchitepéquez", "Suchitepéquez"),
    ("San Andrés Villa Seca", "Suchitepéquez"),
    ("Sayaxché", "Petén"),
    ("El Estor", "Izabal"),
    ("Zunilito", "Suchitepéquez"),
    ("Concepción Chiquirichapa", "Quetzaltenango"),
    ("Cuyotenango", "Suchitepéquez"),
    ("Zaragoza", "Chimaltenango"),
    ("Santa Lucía Milpas Altas", "Sacatepéquez"),
    ("San Miguel Acatán", "Huehuetenango"),
    ("San Agustín Acasaguastlán", "El Progreso"),
    ("Jacaltenango", "Huehuetenango"),
    ("Raxruhá", "Alta Verapaz"),
    ("Santa Rosa de Lima", "Santa Rosa"),
    ("Chicacao", "Suchitepéquez"),
    ("Livingston", "Izabal"),
    ("Nueva Santa Rosa", "Santa Rosa"),
    ("Purulhá", "Baja Verapaz"),
    ("Sumpango", "Sacatepéquez"),
    ("San Luis Jilotepeque", "Jalapa"),
    ("San Juan La Laguna", "Sololá"),
    ("Pueblo Nuevo", "Suchitepéquez"),
    ("Tejutla", "San Marcos"),
    ("Las cruces", "Petén"),
    ("San Gabriel", "Suchitepéquez"),
    ("La Democracia", "Huehuetenango"),
    ("El Progreso", "Jutiapa"),
    ("Santiago Chimaltenango", "Huehuetenango"),
    ("San Francisco", "Petén"),
    ("San Andrés", "Petén"),
    ("Quesada", "Jutiapa"),
    ("San Pedro Ayampuc", "Guatemala"),
    ("Parramos", "Chimaltenango"),
    ("Barberena", "Santa Rosa"),
    ("San Martín Jilotepeque", "Chimaltenango"),
    ("San Andrés Xecul", "Totonicapán"),
    ("Catarina", "San Marcos"),
    ("Champerico", "Retalhuleu"),
    ("San Antonio Aguas Calientes", "Sacatepéquez"),
    ("Zacapa", "Zacapa"),
    ("Santiago Atitlán", "Sololá"),
    ("Morales", "Izabal"),
    ("Granados", "Baja Verapaz"),
    ("Asunción Mita", "Jutiapa"),
    ("La Blanca", "San Marcos"),
    ("Mataquescuintla", "Jalapa"),
    ("Cantel", "Quetzaltenango"),
    ("San José Ojetenam", "San Marcos"),
    ("San Felipe", "Retalhuleu"),
    ("Esquipulas Palo Gordo", "San Marcos"),
    ("Santa Cruz Verapaz", "Alta Verapaz"),
    ("Santa Catarina Barahona", "Sacatepéquez"),
    ("La Gomera", "Escuintla"),
    ("Santa Bárbara", "Huehuetenango"),
    ("San Gaspar Ixchil", "Huehuetenango"),
    ("Cuilco", "Huehuetenango"),
    ("San José El Ídolo", "Suchitepéquez"),
    ("Pueblo Nuevo", "San Marcos"),
    ("San Rafael Pie de La Cuesta", "San Marcos"),
    ("Patzún", "Chimaltenango"),
    ("Concepción Huista", "Huehuetenango"),
    ("Usumatlán", "Zacapa"),
    ("Santa Catarina Palopó", "Sololá"),
    ("Huitán", "Quetzaltenango"),
    ("Palestina de Los Altos", "Quetzaltenango"),
    ("Salamá", "Baja Verapaz"),
    ("Ayutla", "San Marcos"),
    ("Acatenango", "Chimaltenango"),
    ("Santa María Chiquimula", "Totonicapán"),
    ("Chiantla", "Huehuetenango"),
    ("San Juan Bautista", "Suchitepéquez"),
    ("Ciudad Vieja", "Sacatepéquez"),
    ("GUATEMALA", "Guatemala"),
    ("CHINAUTLA", "Guatemala"),
    ("SAN PEDRO AYAMPUC", "Guatemala"),
    ("FRAIJANES", "Guatemala"),
    ("SAN JOSÉ PINULAC", "Guatemala"),
    ("MIXCO", "Guatemala"),
    ("VILLA NUEVA", "Guatemala"),
    ("VILLA CANALES", "Guatemala"),
    ("SANTA CATARINA PINULA", "Guatemala"),
    ("SAN JOSÉ DEL GOLFO", "Guatemala"),
    ("PALENCIA", "Guatemala"),
    ("SAN JUAN SACATEPÉQUEZ", "Guatemala"),
    ("SAN RAIMUNDO", "Guatemala"),
    ("AMATITLÁN", "Guatemala"),
    ("PETAPA", "Guatemala"),
    ("ESCUINTLA", "Escuintla"),
    ("SAN PEDRO SACATEPÉQUEZ", "Guatemala"),
    ("JUTIAPA", "Jutiapa"),
    ("SANTA CRUZ DEL QUICHÉ", "Quiché"),
    ("SAN MIGUEL IXTAHUACÁN", "Huehuetenango"),
    ("SAN PEDRO PINULA", "Jalapa"),
    ("SANTA LUCÍA COTZUMALGUAPA", "Escuintla"),
    ("CHUARRANCHO", "Guatemala"),
    ("PALÍN", "Escuintla"),
    ("ANTIGUA GUATEMALA", "Sacatepéquez"),
    ("SAN JOSÉ", "Escuintla"),
    ("BARBERENA", "Santa Rosa"),
    ("JALAPA", "Jalapa"),
    ("IGNORADO", "Desconocido"),
    ("SAN LUIS JILITEPEQUE", "Jalapa"),
    ("TECPÁN GUATEMALA", "Chimaltenango"),
    ("GUASTATOYA", "El Progreso"),
    ("SAN LUCAS SACATEPÉQUEZ", "Sacatepéquez"),
    ("CABRICÁN", "Quetzaltenango"),
    ("NUEVA SANTA ROSA", "Santa Rosa"),
    ("MAZATENANGO", "Suchitepéquez"),
    ("PUERTO BARRIOS", "Izabal"),
    ("MORAZÁN", "El Progreso"),
    ("LIVINGSTON", "Izabal"),
    ("ZARAGOZA", "Chimaltenango"),
    ("CHIMALTENANGO", "Chimaltenango"),
    ("SAN JOSÉ POAQUIL", "Chimaltenango"),
    ("SAN MARTÍN JILOTEPEQUE", "Chimaltenango"),
    ("COMALAPA", "Chimaltenango"),
    ("PATZICÍA", "Chimaltenango"),
    ("SANTA APOLONIA", "Chimaltenango"),
    ("PATZÚN", "Chimaltenango"),
    ("POCHUTA", "Chimaltenango"),
    ("ACATENANGO", "Chimaltenango"),
    ("YEPOCAPA", "Chimaltenango"),
    ("SAN ANDRÉS ITZAPA", "Chimaltenango"),
    ("PARRAMOS", "Chimaltenango"),
    ("EL TEJAR", "Chimaltenango"),
    ("SACAPULAS", "Quiché"),
    ("SANTA CRUZ BALANYÁ", "Chimaltenango"),
    ("SANTA MARÍA DE JESÚS", "Sacatepéquez"),
    ("SAN FRANCISCO EL ALTO", "Totonicapán"),
    ("SUMPANGO", "Sacatepéquez"),
    ("LA DEMOCRACIA", "Escuintla"),
    ("SIQUINALÁ", "Escuintla"),
    ("MASAGUA", "Escuintla"),
    ("TIQUISATE", "Escuintla"),
    ("LA GOMERA", "Escuintla"),
    ("GUANAGAZAPA", "Escuintla"),
    ("IZTAPA", "Escuintla"),
    ("SAN VICENTE PACAYA", "Escuintla"),
    ("NUEVA CONCEPCIÓN", "Escuintla"),
    ("NUEVO PROGRESO", "San Marcos"),
    ("SANTA ROSA DE LIMA", "Santa Rosa"),
    ("CUILAPA", "Santa Rosa"),
    ("GUAZACAPÁN", "Santa Rosa"),
    ("ORATORIO", "Santa Rosa"),
    ("PUEBLO NUEVO VIÑAS", "Santa Rosa"),
    ("SANTA MARÍA IXHUATÁN", "Santa Rosa"),
    ("SANTA CRUZ NARANJO", "Santa Rosa"),
    ("CASILLAS", "Santa Rosa"),
    ("SAN RAFAEL LAS FLORES", "Santa Rosa"),
    ("CHIQUIMULILLA", "Santa Rosa"),
    ("TAXISCO", "Santa Rosa"),
    ("SAN JUAN TECUACO", "Santa Rosa"),
    ("MATAQUESCUINTLA", "Jalapa"),
    ("SAN JOSÉ CHACAYÁ", "Sololá"),
    ("NAHUALÁ", "Sololá"),
    ("SANTA CATARINA IXTAHUACÁN", "Sololá"),
    ("CONCEPCIÓN", "Sololá"),
    ("SAN ANDRÉS SEMETABAJ", "Sololá"),
    ("SANTA CATARINA PALOPÓ", "Sololá"),
    ("SANTA CRUZ LA LAGUNA", "Sololá"),
    ("SAN JUAN LA LAGUNA", "Sololá"),
    ("SANTIAGO ATITLÁN", "Sololá"),
    ("SANTA CLARA LA LAGUNA", "Sololá"),
    ("SAN LUCAS TOLIMÁN", "Sololá"),
    ("SOLOLÁ", "Sololá"),
    ("SANTA LUCÍA UTATLÁN", "Sololá"),
    ("PANAJACHEL", "Sololá"),
    ("SAN ANTONIO PALOPÓ", "Sololá"),
    ("SAN PEDRO LA LAGUNA", "Sololá"),
    ("SAN PABLO LA LAGUNA", "Sololá"),
    ("SAN MARCOS LA LAGUNA", "Sololá"),
    ("TOTONICAPÁN", "Totonicapán"),
    ("SAN ANDRÉS XECUL", "Totonicapán"),
    ("SAN CRISTÓBAL TOTONICAPÁN", "Totonicapán"),
    ("MONOSTENANGO", "Totonicapán"),
    ("SAN BARTOLO", "Totonicapán"),
    ("SAN LUCÍA LA REFORMA", "Totonicapán"),
    ("SANTA MARÍA CHIQUIMULA", "Totonicapán"),
    ("QUETZALTENANGO", "Quetzaltenango"),
    ("SAN FRANCISCO ZAPOTITLÁN", "Quetzaltenango"),
    ("CAJOLÁ", "Quetzaltenango"),
    ("SAN CARLOS SIJA", "Quetzaltenango"),
    ("SAN MATEO", "Quetzaltenango"),
    ("SALCAJÁ", "Quetzaltenango"),
    ("OLINTEPEQUE", "Quetzaltenango"),
    ("SIBILIA", "Quetzaltenango"),
    ("OSTUNCALCO", "Quetzaltenango"),
    ("CONCEPCIÓN CHIQUIRICHAPA", "Quetzaltenango"),
    ("SAN MARTÍN SACATEPÉQUEZ", "Quetzaltenango"),
    ("ALMOLONGA", "Quetzaltenango"),
    ("CANTEL", "Quetzaltenango"),
    ("ZUNIL", "Quetzaltenango"),
    ("HUITÁN", "Quetzaltenango"),
    ("COLOMBA", "Quetzaltenango"),
    ("EL PALMAR", "Quetzaltenango"),
    ("COATEPEQUE", "Quetzaltenango"),
    ("GÉNOVA", "Quetzaltenango"),
    ("FLORES COSTA CUCA", "Quetzaltenango"),
    ("PAJAPITA", "San Marcos"),
    ("LA ESPERANZA", "Quetzaltenango"),
    ("EL QUETZAL", "San Marcos"),
    ("PALESTINA DE LOS ALTOS", "Quetzaltenango"),
    ("SAN MIGUEL SIGÜILÁ", "Quetzaltenango"),
    ("SAN ANTONIO SACATEPÉQUEZ", "Sacatepéquez"),
    ("SAN JUAN COTZAL", "Quiché"),
    ("SAN FRANCISCO LA UNIÓN", "Quiché"),
    ("CUYOTENANGO", "Suchitepéquez"),
    ("SAN SEBASTIÁN", "Quetzaltenango"),
    ("CHAPERICO", "Retalhuleu"),
    ("RETALHULEU", "Retalhuleu"),
    ("NUEVO SAN CARLOS", "Retalhuleu"),
    ("SAN ANDRÉS VILLA SECA", "Suchitepéquez"),
    ("SAN MARTÍN ZAPOTITLÁN", "Retalhuleu"),
    ("SAN FELIPE", "Retalhuleu"),
    ("EL ASINTAL", "Retalhuleu"),
    ("SANTA CRUZ MULUÁ", "Retalhuleu"),
    ("HUEHUETENANGO", "Huehuetenango"),
    ("SAN MARCOS", "San Marcos"),
    ("SAN RAFAEL PIÉ DE LA CUESTA", "San Marcos"),
    ("COMITANCILLO", "San Marcos"),
    ("CONCEPCIÓN TUTUAPA", "San Marcos"),
    ("TACANÁ", "San Marcos"),
    ("SIBINAL", "San Marcos"),
    ("TAJUMULCO", "San Marcos"),
    ("TEJUTLA", "San Marcos"),
    ("EL TUMBADOR", "San Marcos"),
    ("EL RODEO", "San Marcos"),
    ("MALACATÁN", "San Marcos"),
    ("OCÓS", "San Marcos"),
    ("SAN PABLO", "San Marcos"),
    ("LA REFORMA", "San Marcos"),
    ("IXCHIGUÁN", "San Marcos"),
    ("SIPACAPA", "San Marcos"),
    ("ESQUIPULAS PALO GORDO", "San Marcos"),
    ("RÍO BLANCO", "San Marcos"),
    ("SAN LORENZO", "San Marcos"),
    ("LA BLANCA", "San Marcos"),
    ("CATARINA", "San Marcos"),
    ("SAN JOSÉ OJETENAM", "San Marcos"),
    ("SAN CRISTÓBAL CUCHO", "San Marcos"),
    ("AYUTLA", "San Marcos"),
    ("CHIANTLA", "Huehuetenango"),
    ("SANTA BÁRBARA", "Huehuetenango"),
    ("AGUACATÁN", "Huehuetenango"),
    ("LA DEMOCRACIA", "Huehuetenango"),
    ("COLOTENANGO", "Huehuetenango"),
    ("SAN SEBASTIÁN HUEHUETENANGO", "Huehuetenango"),
    ("TECTITÁN", "Huehuetenango"),
    ("MALACATANCITO", "Huehuetenango"),
    ("CUILCO", "Huehuetenango"),
    ("SAN PEDRO NECTA", "Huehuetenango"),
    ("SOLOMA", "Huehuetenango"),
    ("LA LIBERTAD", "Huehuetenango"),
    ("SAN MIGUEL ACATÁN", "Huehuetenango"),
    ("SAN RAFAEL LA INDEPENDENCIA", "Huehuetenango"),
    ("TODOS SANTOS CUCHUMATÁN", "Huehuetenango"),
    ("BARILLAS", "Huehuetenango"),
    ("SANTA EULALIA", "Huehuetenango"),
    ("SAN JUAN IXCOY", "Huehuetenango"),
    ("SAN ANTONIO HUISTA", "Huehuetenango"),
    ("SAN SEBASTIÁN COATÁN", "Huehuetenango"),
    ("SAN GASPAR IXCHIL", "Huehuetenango"),
    ("SAN JUAN ATITÁN", "Huehuetenango"),
    ("NENTÓN", "Huehuetenango"),
    ("JACALTENANGO", "Huehuetenango"),
    ("IXTAHUCÁN", "Huehuetenango"),
    ("SAN RAFAEL PETZAL", "Huehuetenango"),
    ("PETATAN", "Huehuetenango"),
    ("SANTIAGO CHIMALTENANGO", "Huehuetenango"),
    ("SANTA ANA HUISTA", "Huehuetenango"),
    ("CONCEPCIÓN HUISTA", "Huehuetenango"),
    ("IXCÁN", "Quiché"),
    ("ZACUALPA", "Quiché"),
    ("CHAJUL", "Quiché"),
    ("CHICHICASTENANGO", "Quiché"),
    ("PATZITÉ", "Quiché"),
    ("SAN ANTONIO ILOTENANGO", "Quiché"),
    ("CUNÉN", "Quiché"),
    ("JOYABAJ", "Quiché"),
    ("NEBAJ", "Quiché"),
    ("SAN ANDRÉS SAJCABAJÁ", "Quiché"),
    ("SAN BARTALOMÉ JOCOTENANGO", "Quiché"),
    ("CANILLÁ", "Quiché"),
    ("CHICAMÁN", "Quiché"),
    ("PACHALUM", "Quiché"),
    ("CHICHÉ", "Quiché"),
    ("CHINIQUE", "Quiché"),
    ("COBÁN", "Alta Verapaz"),
    ("USPANTÁN", "Quiché"),
    ("SALAMÁ", "Baja Verapaz"),
    ("SAN MIGUEL CHICAJ", "Baja Verapaz"),
    ("RABINAL", "Baja Verapaz"),
    ("CUBULCO", "Baja Verapaz"),
    ("GRANADOS", "Baja Verapaz"),
    ("SAN JERÓNIMO", "Baja Verapaz"),
    ("PURULHÁ", "Baja Verapaz"),
    ("EL CHOL", "Baja Verapaz"),
    ("SAN PEDRO CARCHÁ", "Alta Verapaz"),
    ("SAN JUAN CHAMELCO", "Alta Verapaz"),
    ("SANTA CRUZ VERAPAZ", "Alta Verapaz"),
    ("SAN CRISTÓBAL VERAPAZ", "Alta Verapaz"),
    ("TACTIC", "Alta Verapaz"),
    ("TAMAHÚ", "Alta Verapaz"),
    ("TUCURÚ", "Alta Verapaz"),
    ("PANZÓS", "Alta Verapaz"),
    ("SENAHÚ", "Alta Verapaz"),
    ("LANQUÍN", "Alta Verapaz"),
    ("CHISEC", "Alta Verapaz"),
    ("CHAHAL", "Alta Verapaz"),
    ("FRAY BARTOLOMÉ DE LAS CASAS", "Alta Verapaz"),
    ("SAN CATALINA LA TINTA", "Alta Verapaz"),
    ("RAXRUJÀ", "Alta Verapaz"),
    ("CAHABÓN", "Alta Verapaz"),
    ("SAYAXCHÉ", "Petén"),
    ("SAN LUIS", "Petén"),
    ("FLORES", "Petén"),
    ("SAN BENITO", "Petén"),
    ("SAN ANDRÉS", "Petén"),
    ("SAN FRANCISCO", "Petén"),
    ("DOLORES", "Petén"),
    ("EL CHAL", "Petén"),
    ("MELCHOR DE MENCOS", "Petén"),
    ("POPTÚN", "Petén"),
    ("LAS CRUCES", "Petén"),
    ("SANTA ANA", "Petén"),
    ("GUALÁN", "Zacapa"),
    ("ASUNCIÓN MITA", "Jutiapa"),
    ("SAN ANTONIO LA PAZ", "El Progreso"),
    ("EL JÍCARO", "El Progreso"),
    ("SAN AGUSTÍN ACASAGUASTLAN", "El Progreso"),
    ("SANARATE", "El Progreso"),
    ("SAN CRISTÓBAL ACASAGUASTLAN", "El Progreso"),
    ("SANSARE", "El Progreso"),
    ("AGUA BLANCA", "Jutiapa"),
    ("JOCOTENANGO", "Sacatepéquez"),
    ("PASTORES", "Sacatepéquez"),
    ("SANTO DOMINGO XENACOJ", "Sacatepéquez"),
    ("SANTIAGO SACATEPÉQUEZ", "Sacatepéquez"),
    ("SAN BARTOLOMÉ MILPAS ALTAS", "Sacatepéquez"),
    ("SANTA LUCÍA MILPAS ALTAS", "Sacatepéquez"),
    ("MAGDALENA MILPAS ALTAS", "Sacatepéquez"),
    ("CIUDAD VIEJA", "Sacatepéquez"),
    ("SAN MIGUEL DUEÑAS", "Sacatepéquez"),
    ("ALOTENANGO", "Sacatepéquez"),
    ("SANTA CATARINA BARAHONA", "Sacatepéquez"),
    ("SAMAYAC", "Suchitepéquez"),
    ("SANTO DOMINGO SUCHITEPÉQUEZ", "Suchitepéquez"),
    ("SAN ANTONIO SUCHITEPÉQUEZ", "Suchitepéquez"),
    ("ZUNILITO", "Suchitepéquez"),
    ("SAN JOSÉ LA MÁQUINA", "Suchitepéquez"),
    ("SAN BERNARDINO", "Suchitepéquez"),
    ("SAN JOSÉ EL ÍDOLO", "Suchitepéquez"),
    ("SAN PABLO JOCOPILAS", "Suchitepéquez"),
    ("SAN GABRIEL", "Suchitepéquez"),
    ("CHICACAO", "Suchitepéquez"),
    ("SAN JUAN BAUTISTA", "Suchitepéquez"),
    ("PATULUL", "Suchitepéquez"),
    ("SANTO TOMÁS LA UNIÓN", "Suchitepéquez"),
    ("PUEBLO NUEVO", "Suchitepéquez"),
    ("RÍO BRAVO", "Suchitepéquez"),
    ("MORALES", "Izabal"),
    ("EL ESTOR", "Izabal"),
    ("ZACAPA", "Zacapa"),
    ("TECULUTÁN", "Zacapa"),
    ("SAN JORGE", "Zacapa"),
    ("ESTANZUELA", "Zacapa"),
    ("RÍO HONDO", "Zacapa"),
    ("HUITÉ", "Zacapa"),
    ("ESQUIPULAS", "Chiquimula"),
    ("SAN JACINTO", "Chiquimula"),
    ("CHIQUIMULA", "Chiquimula"),
    ("OLOPA", "Chiquimula"),
    ("SAN JUAN ERMITA", "Chiquimula"),
    ("JOCOTÁN", "Chiquimula"),
    ("CONCEPCIÓN LAS MINAS", "Chiquimula"),
    ("QUETZALTEPEQUE", "Chiquimula"),
    ("IPALA", "Chiquimula"),
    ("MONJAS", "Jalapa"),
    ("SAN MANUEL CHAPARRÓN", "Jalapa"),
    ("CONGUACO", "Jutiapa"),
    ("SANTA CATARINA MITA", "Jutiapa"),
    ("YUPILTEPEQUE", "Jutiapa"),
    ("COMAPA", "Jutiapa"),
    ("JALPATAGUA", "Jutiapa"),
    ("EL PROGRESO", "Jutiapa"),
    ("ATESCATEMPA", "Jutiapa"),
    ("JEREZ", "Jutiapa"),
    ("ZAPOTITLÁN", "Jutiapa"),
    ("PASACO", "Jutiapa"),
    ("QUEZADA", "Jutiapa"),
    ("SAN MIGUEL PANÁN", "Jutiapa"),
    ("LOS AMATES", "Izabal"),
    ("USUMATLÁN", "Zacapa"),
    ("SAN DIEGO", "Zacapa"),
    ("LA UNIÓN", "Zacapa"),
    ("SAN JOSÉ LA ARADA", "Chiquimula"),
    ("EL ADELANTO", "Jutiapa"),
    ("SAN JOSÉ ACATEMPA", "Jutiapa"),
    ("SAN ANTONIO AGUAS CALIENTES", "Sacatepéquez"),
    ("CABAÑAS", "Zacapa"),
    ("CAMOTÁN", "Chiquimula"),
    ("MOYUTA", "Jutiapa"),
    ("SAN MATEO IXTATÁN", "Huehuetenango"),
    ("La Tinta", "Alta Verapaz"),
    ("Fray Bartolomé de las Casas", "Alta Verapaz"),
    ("Guanazapa", "Escuintla"),
    ("San Joseé La Máquina", "Suchitepéquez"),
    ("San Cristobal Acasaguastlán", "El Progreso"),
    ("Zapotitlán", "Jutiapa"),
    ("San Francisco el Alto", "Totonicapán"),
    ("Santa Lucia Utatlán", "Sololá"),
    ("San Antonio la Paz", "El Progreso"),
    ("San Francisco la Unión", "Quiché"),
    ("Santiago Sactepéquez", "Sacatepéquez"),
    ("El Jícaro", "El Progreso"),
    ("Santa Catalina Ixtahuacán", "Sololá"),
    ("Santo Tomás la Unión", "Suchitepéquez"),
    ("San Andres Semetabaj", "Sololá"),
    ("Melchor", "Petén"),
    ("San Rafael la Independencia", "Huehuetenango"),
    ("Santiago Chimaltenanango", "Huehuetenango"),
    ("Poptún", "Petén"),
    ("San Luis", "Petén"),
    ("Comitancillo", "San Marcos"),
    ("Dolores", "Petén"),
    ("San Antonio Sacatepéquez", "San Marcos"),
    ("Nuevo San Carlos", "Retalhuleu"),
    ("San Miguel Ixtahuacán", "San Marcos"),
    ("San Lucas Sacatepéquez", "Sacatepéquez"),
    ("Rabinal", "Baja Verapaz"),
    ("San Miguel Chicaj", "Baja Verapaz"),
    ("San José el Golfo", "Guatemala"),
    ("San Raimundo", "Guatemala"),
    ("Petapa", "Guatemala"),
    ("Guatemala", "Guatemala"),
    ("Alotenango", "Sacatepéquez"),
    ("Chimaltenango", "Chimaltenango"),
    ("Yepocapa", "Chimaltenango"),
    ("San Rafael las Flores", "Santa Rosa"),
    ("Santiago Atitlán", "Sololá"),
    ("San Cristobal Totonicapán", "Totonicapán"),
    ("Santa Lucía La Reforma", "Totonicapán"),
    ("Salcajá", "Quetzaltenango"),
    ("Olintepeque", "Quetzaltenango"),
    ("Ostuncalco", "Quetzaltenango"),
    ("Colomba", "Quetzaltenango"),
    ("Palestina de los Altos", "Quetzaltenango"),
    ("San José El Idolo", "Suchitepéquez"),
    ("Patulul", "Suchitepéquez"),
    ("Santo Tomás La Unión", "Suchitepéquez"),
    ("Rio Bravo", "Escuintla"),
    ("San Rafael Pié de La Cuesta", "San Marcos"),
    ("San Marcos", "San Marcos"),
    ("El Rodeo", "San Marcos"),
    ("Ocós", "San Marcos"),
    ("Ixtahuacán", "Huehuetenango"),
    ("San Antonio Huista", "Huehuetenango"),
    ("Chichicastenango", "Quiché"),
    ("Uspantán", "Quiché"),
    ("Ixcán", "Quiché"),
    ("Nebaj", "Quiché"),
    ("Tucurú", "Alta Verapaz"),
    ("Lanquín", "Alta Verapaz"),
    ("Cahabón", "Alta Verapaz"),
    ("Santa Catalina la Tinta", "Alta Verapaz"),
    ("Jocotán", "Chiquimula"),
    ("Quetzaltepeque", "Chiquimula"),
    ("San José Acatempa", "Jutiapa"),
    ("Villa Nueva", "Guatemala"),
    ("Comalapa", "Chimaltenango"),
    ("Pochuta", "Chimaltenango"),
    ("Santa Lucía Cotzumalguapa", "Escuintla"),
    ("San Bartolo", "Totonicapán"),
    ("Quetzaltenango", "Quetzaltenango"),
    ("Mazatenango", "Suchitepéquez"),
    ("El Asintal", "Retalhuleu"),
    ("Rio Blanco", "San Marcos"),
    ("Barillas", "Huehuetenango"),
    ("Santa Cruz del Quiché", "Quiché"),
    ("El Chol", "Baja Verapaz"),
    ("Chiquimula", "Chiquimula"),
    ("San Juan Erminta", "Chiquimula"),
    ("San Agustín Acasaguastlán", "El Progreso"),
    ("Totonicapán", "Totonicapán"),
    ("San Francisco La Unión", "Quiché"),
    ("Coatepeque", "Quetzaltenango"),
    ("Unión Cantinil", "Huehuetenango"),
    ("Chinique", "Quiché"),
    ("Antigua Guatemala", "Sacatepéquez"),
    ("Amatitlán", "Guatemala"),
    ("San Antonio Sacatepéquez", "San Marcos"),
    ("Huehuetenango", "Huehuetenango"),
    ("Cobán", "Alta Verapaz"),
    ("Jalapa", "Jalapa"),
    ("Nueva Concepción", "Escuintla"),
    ("San Francisco El Alto", "Totonicapán"),
    ("Soloma", "Huehuetenango"),
    ("Río Hondo", "Zacapa"),
    ("Jutiapa", "Jutiapa"),
    ("Jerez", "Jutiapa"),
    ("Malacatán", "San Marcos"),
    ("San Lucas Toliman", "Sololá"),
    ("San Mateo Ixtatán", "Huehuetenango"),
    ("Tamahú", "Alta Verapaz"),
    ("Flores", "Petén"),
    ("Camotán", "Chiquimula"),
    ("San José Chacayá", "Sololá"),
    ("Cabricán", "Quetzaltenango"),
    ("El Tumbador", "San Marcos"),
    ("San Juan Sacatepéquez", "Guatemala"),
    ("Villa Canales", "Guatemala"),
    ("Cajolá", "Quetzaltenango"),
    ("La Esperanza", "Quetzaltenango"),
    ("El Quetzal", "San Marcos"),
    ("Sibinal", "San Marcos"),
    ("Patzún", "Chimaltenango"),
    ("Parramos", "Chimaltenango"),
    ("Flores Costa Cuca", "Quetzaltenango"),
    ("Cuilapa", "Santa Rosa"),
    ("Colotenango", "Huehuetenango"),
    ("Chajul", "Quiché"),
    ("El Estor", "Izabal"),
    ("Mixco", "Guatemala"),
    ("San Andrés Itzapa", "Chimaltenango"),
    ("Palín", "Escuintla"),
    ("Casillas", "Santa Rosa"),
    ("Pajapita", "San Marcos"),
    ("San Andrés Xecul", "Totonicapán"),
    ("La Democracia", "Escuintla"),
    ("Sololá", "Sololá"),
    ("Esquipulas", "Chiquimula"),
    ("San Carlos Sija", "Quetzaltenango"),
    ("Chicamán", "Quiché"),
    ("Ayutla", "San Marcos"),
    ("Salamá", "Baja Verapaz"),
    ("Concepción Chiquirichapa", "Quetzaltenango"),
    ("Tecpán Guatemala", "Chimaltenango"),
    ("San Antonio Ilotenango", "Quiché"),
    ("Momostenango", "Totonicapán"),
    ("San Pedro Necta", "Huehuetenango"),
    ("Pastores", "Sacatepéquez"),
    ("Cantel", "Quetzaltenango"),
    ("Chiché", "Quiché"),
    ("Siquinalá", "Escuintla"),
    ("Chuarrancho", "Guatemala"),
    ("San Bernardino", "Suchitepéquez"),
    ("Rabinal", "Baja Verapaz"),
    ("San Martín Zapotitlán", "Retalhuleu"),
    ("Santa Catarina Pinula", "Guatemala"),
    ("Chicacao", "Suchitepéquez"),
    ("San Cristóbal Verapaz", "Alta Verapaz"),
    ("San José Poaquil", "Chimaltenango"),
    ("San Vicente Pacaya", "Escuintla"),
    ("San Andrés Sajcabajá", "Quiché"),
    ("San Antonio Suchitepéquez", "Suchitepéquez"),
    ("Taxisco", "Santa Rosa"),
    ("Los Amates", "Izabal"),
    ("Palencia", "Guatemala"),
    ("Fraijanes", "Guatemala"),
    ("San Antonio La Paz", "El Progreso"),
    ("Santo Domingo Xenacoj", "Sacatepéquez"),
    ("Patzicía", "Chimaltenango"),
    ("La Gomera", "Escuintla"),
    ("San José", "Escuintla"),
    ("Santa María Chiquimula", "Totonicapán"),
    ("Sibilia", "Quetzaltenango"),
    ("San Miguel Siguilá", "Quetzaltenango"),
    ("San Francisco Zapotitlán", "Suchitepéquez"),
    ("Santo Domingo Suchitepéquez", "Suchitepéquez"),
    ("Samayac", "Suchitepéquez"),
    ("San Pablo Jocopilas", "Suchitepéquez"),
    ("Santa Bárbara", "Huehuetenango"),
    ("Retalhuleu", "Retalhuleu"),
    ("San Pedro Sacatepéquez", "Guatemala"),
    ("Concepción Tutuapa", "San Marcos"),
    ("Tejutla", "San Marcos"),
    ("Catarina", "San Marcos"),
    ("San Lorenzo", "San Marcos"),
    ("San Miguel Acatán", "Huehuetenango"),
    ("San Juan Atitán", "Huehuetenango"),
    ("Aguacatán", "Huehuetenango"),
    ("San Rafael Petzal", "Huehuetenango"),
    ("Santiago Chimaltenango", "Huehuetenango"),
    ("Santa Ana Huista", "Huehuetenango"),
    ("Zacualpa", "Quiché"),
    ("San Pedro Jocopilas", "Quiché"),
    ("Cunén", "Quiché"),
    ("Joyabaj", "Quiché"),
    ("Sacapulas", "Quiché"),
    ("Canillá", "Quiché"),
    ("Purulhá", "Baja Verapaz"),
    ("Tactic", "Alta Verapaz"),
    ("San Pedro Carchá", "Alta Verapaz"),
    ("Chisec", "Alta Verapaz"),
    ("Chahal", "Alta Verapaz"),
    ("La Unión", "Zacapa"),
    ("Olopa", "Chiquimula"),
    ("San Jacinto", "Chiquimula"),
    ("Asunción Mita", "Jutiapa"),
    ("Chinautla", "Guatemala"),
    ("San Martín Sacatepéquez", "Quetzaltenango"),
    ("Guastatoya", "El Progreso"),
    ("Santa Eulalia", "Huehuetenango"),
    ("Santa Catarina Ixtahuacán", "Sololá"),
    ("San Andrés Semetabaj", "Sololá"),
    ("San Juan Bautista", "Suchitepéquez"),
    ("Pueblo Nuevo", "Suchitepéquez"),
    ("Cuilco", "Huehuetenango"),
    ("San Sebastián Huehuetenango", "Huehuetenango"),
    ("Moyuta", "Jutiapa"),
    ("Sayaxché", "Petén"),
    ("Ipala", "Chiquimula"),
    ("San Juan Cotzal", "Quiché"),
    ("La Libertad", "Petén"),
    ("Las cruces", "Petén"),
    ("Sumpango", "Sacatepéquez"),
    ("Escuintla", "Escuintla"),
    ("San Mateo", "Quetzaltenango"),
    ("Mataquescuintla", "Jalapa"),
    ("San Pedro Ayampuc", "Guatemala"),
    ("Zaragoza", "Chimaltenango"),
    ("Panzós", "Alta Verapaz"),
    ("San Francisco", "Petén"),
    ("San Martín Jilotepeque", "Chimaltenango"),
    ("Pueblo Nuevo Viñas", "Santa Rosa"),
    ("Sipacapa", "San Marcos"),
    ("San Sebastián", "Quetzaltenango"),
    ("Patzité", "Quiché"),
    ("Santa Cruz Verapaz", "Alta Verapaz"),
    ("San Benito", "Petén"),
    ("San Miguel Panán", "Jutiapa"),
    ("San Juan Chamelco", "Alta Verapaz"),
    ("Raxruhá", "Alta Verapaz"),
    ("Almolonga", "Quetzaltenango"),
    ("San Rafael La Independencia", "Huehuetenango"),
    ("Nueva Santa Rosa", "Santa Rosa"),
    ("San Miguel Chicaj", "Baja Verapaz"),
    ("Pasaco", "Jutiapa"),
    ("San Gabriel", "Suchitepéquez"),
    ("Cubulco", "Baja Verapaz"),
    ("Santa Lucía Milpas Altas", "Sacatepéquez"),
    ("Granados", "Baja Verapaz"),
    ("Ciudad Vieja", "Sacatepéquez"),
    ("San Miguel Dueñas", "Sacatepéquez"),
    ("Santiago Sacatepéquez", "Sacatepéquez"),
    ("San Antonio Palopó", "Sololá"),
    ("Santa Cruz Balanyá", "Chimaltenango"),
    ("Poptún", "Petén"),
    ("Santa Cruz Naranjo", "Santa Rosa"),
    ("San José El Ídolo", "Suchitepéquez"),
    ("Pueblo Nuevo", "San Marcos"),
    ("San Rafael Pie de La Cuesta", "San Marcos"),
    ("San Pablo", "San Marcos"),
    ("Usumatlán", "Zacapa"),
    ("Malacatancito", "Huehuetenango"),
    ("Magdalena Milpas Altas", "Sacatepéquez"),
    ("Acatenango", "Chimaltenango"),
    ("Concepción Las Minas", "Chiquimula"),
    ("Yupiltepeque", "Jutiapa"),
    ("Santa Catarina Barahona", "Sacatepéquez"),
    ("Quesada", "Jutiapa"),
    ("El Palmar", "Quetzaltenango"),
    ("El Tejar", "Chimaltenango"),
    ("Fray Bartolomé de Las Casas", "Alta Verapaz"),
    ("Concepción", "Sololá"),
    ("Comitancillo", "San Marcos"),
    ("La Reforma", "San Marcos"),
    ("Nentón", "Huehuetenango"),
    ("San Antonio Aguas Calientes", "Sacatepéquez"),
    ("Atescatempa", "Jutiapa"),
    ("GUATEMALA", "Guatemala"),
    ("VILLA NUEVA", "Guatemala"),
    ("SANTA CATARINA PINULA", "Guatemala"),
    ("SAN JOSÉ PINULAC", "Guatemala"),
    ("SAN JOSÉ DEL GOLFO", "Guatemala"),
    ("PALENCIA", "Guatemala"),
    ("CHINAUTLA", "Guatemala"),
    ("SAN PEDRO AYAMPUC", "Guatemala"),
    ("MIXCO", "Guatemala"),
    ("SAN JUAN SACATEPÉQUEZ", "Guatemala"),
    ("SAN RAIMUNDO", "Guatemala"),
    ("FRAIJANES", "Guatemala"),
    ("AMATITLÁN", "Guatemala"),
    ("VILLA CANALES", "Guatemala"),
    ("PETAPA", "Guatemala"),
    ("CHUARRANCHO", "Guatemala"),
    ("SAN PEDRO SACATEPÉQUEZ", "Guatemala"),
    ("CHIMALTENANGO", "Chimaltenango"),
    ("SAN JOSÉ POAQUIL", "Chimaltenango"),
    ("SAN MARTÍN JILOTEPEQUE", "Chimaltenango"),
    ("COMALAPA", "Chimaltenango"),
    ("SANTA APOLONIA", "Chimaltenango"),
    ("TECPÁN GUATEMALA", "Chimaltenango"),
    ("PATZÚN", "Chimaltenango"),
    ("POCHUTA", "Chimaltenango"),
    ("PATZICÍA", "Chimaltenango"),
    ("ACATENANGO", "Chimaltenango"),
    ("YEPOCAPA", "Chimaltenango"),
    ("SAN ANDRÉS ITZAPA", "Chimaltenango"),
    ("PARRAMOS", "Chimaltenango"),
    ("ZARAGOZA", "Chimaltenango"),
    ("EL TEJAR", "Chimaltenango"),
    ("SANTA CRUZ BALANYÁ", "Chimaltenango"),
    ("SAN MARTÍN SACATEPÉQUEZ", "Quetzaltenango"),
    ("ESCUINTLA", "Escuintla"),
    ("SANTA LUCÍA COTZUMALGUAPA", "Escuintla"),
    ("LA DEMOCRACIA", "Escuintla"),
    ("SIQUINALÁ", "Escuintla"),
    ("MASAGUA", "Escuintla"),
    ("TIQUISATE", "Escuintla"),
    ("LA GOMERA", "Escuintla"),
    ("GUANAGAZAPA", "Escuintla"),
    ("SAN JOSÉ", "Escuintla"),
    ("IZTAPA", "Escuintla"),
    ("PALÍN", "Escuintla"),
    ("SAN VICENTE PACAYA", "Escuintla"),
    ("NUEVA CONCEPCIÓN", "Escuintla"),
    ("CUILAPA", "Santa Rosa"),
    ("BARBERENA", "Santa Rosa"),
    ("SANTA ROSA DE LIMA", "Santa Rosa"),
    ("CASILLAS", "Santa Rosa"),
    ("SAN RAFAEL LAS FLORES", "Santa Rosa"),
    ("ORATORIO", "Santa Rosa"),
    ("CHIQUIMULILLA", "Santa Rosa"),
    ("SANTA MARÍA IXHUATÁN", "Santa Rosa"),
    ("SANTA CRUZ NARANJO", "Santa Rosa"),
    ("PUEBLO NUEVO VIÑAS", "Santa Rosa"),
    ("NUEVA SANTA ROSA", "Santa Rosa"),
    ("TAXISCO", "Santa Rosa"),
    ("GUAZACAPÁN", "Santa Rosa"),
    ("SAN JOSÉ CHACAYÁ", "Sololá"),
    ("NAHUALÁ", "Sololá"),
    ("SANTA CATARINA IXTAHUCÁN", "Sololá"),
    ("CONCEPCIÓN", "Sololá"),
    ("SAN ANDRÉS SEMETABAJ", "Sololá"),
    ("SANTA CATARINA PALOPÓ", "Sololá"),
    ("SANTA CRUZ LA LAGUNA", "Sololá"),
    ("SAN JUAN LA LAGUNA", "Sololá"),
    ("SANTIAGO ATITLÁN", "Sololá"),
    ("SANTA CLARA LA LAGUNA", "Sololá"),
    ("SAN LUCAS TOLIMÁN", "Sololá"),
    ("SOLOLÁ", "Sololá"),
    ("SANTA LUCÍA UTATLÁN", "Sololá"),
    ("PANAJACHEL", "Sololá"),
    ("SAN ANTONIO PALOPÓ", "Sololá"),
    ("SAN PEDRO LA LAGUNA", "Sololá"),
    ("SAN PABLO LA LAGUNA", "Sololá"),
    ("SAN MARCOS LA LAGUNA", "Sololá"),
    ("TOTONICAPÁN", "Totonicapán"),
    ("SAN CRISTÓBAL TOTONICAPÁN", "Totonicapán"),
    ("SAN FRANCISCO EL ALTO", "Totonicapán"),
    ("MONOSTENANGO", "Totonicapán"),
    ("SAN BARTOLO", "Totonicapán"),
    ("SAN ANDRÉS XECUL", "Totonicapán"),
    ("SAN LUCÍA LA REFORMA", "Totonicapán"),
    ("SANTA MARÍA CHIQUIMULA", "Totonicapán"),
    ("QUETZALTENANGO", "Quetzaltenango"),
    ("SALCAJÁ", "Quetzaltenango"),
    ("OLINTEPEQUE", "Quetzaltenango"),
    ("SIBILIA", "Quetzaltenango"),
    ("CABRICÁN", "Quetzaltenango"),
    ("OSTUNCALCO", "Quetzaltenango"),
    ("SAN MATEO", "Quetzaltenango"),
    ("CONCEPCIÓN CHIQUIRICHAPA", "Quetzaltenango"),
    ("ALMOLONGA", "Quetzaltenango"),
    ("CANTEL", "Quetzaltenango"),
    ("HUITÁN", "Quetzaltenango"),
    ("COLOMBA", "Quetzaltenango"),
    ("EL PALMAR", "Quetzaltenango"),
    ("COATEPEQUE", "Quetzaltenango"),
    ("GÉNOVA", "Quetzaltenango"),
    ("FLORES COSTA CUCA", "Quetzaltenango"),
    ("LA ESPERANZA", "Quetzaltenango"),
    ("CAJOLÁ", "Quetzaltenango"),
    ("PALESTINA DE LOS ALTOS", "Quetzaltenango"),
    ("SAN MIGUEL SIGÜILÁ", "Quetzaltenango"),
    ("SAN FRANCISCO LA UNIÓN", "Quiché"),
    ("SAN CARLOS SIJA", "Quetzaltenango"),
    ("RETALHULEU", "Retalhuleu"),
    ("SAN SEBASTIÁN", "Quetzaltenango"),
    ("SAN MARTÍN ZAPOTITLÁN", "Retalhuleu"),
    ("SAN FELIPE", "Retalhuleu"),
    ("SAN ANDRÉS VILLA SECA", "Suchitepéquez"),
    ("CHAPERICO", "Retalhuleu"),
    ("NUEVO SAN CARLOS", "Retalhuleu"),
    ("EL ASINTAL", "Retalhuleu"),
    ("SANTA CRUZ MULUÁ", "Retalhuleu"),
    ("SAN MARCOS", "San Marcos"),
    ("SAN ANTONIO SACATEPÉQUEZ", "San Marcos"),
    ("COMITANCILLO", "San Marcos"),
    ("SAN MIGUEL IXTAHUACÁN", "San Marcos"),
    ("CONCEPCIÓN TUTUAPA", "San Marcos"),
    ("TACANÁ", "San Marcos"),
    ("SIBINAL", "San Marcos"),
    ("TAJUMULCO", "San Marcos"),
    ("TEJUTLA", "San Marcos"),
    ("SAN RAFAEL PIÉ DE LA CUESTA", "San Marcos"),
    ("EL TUMBADOR", "San Marcos"),
    ("EL RODEO", "San Marcos"),
    ("MALACATÁN", "San Marcos"),
    ("OCÓS", "San Marcos"),
    ("SAN PABLO", "San Marcos"),
    ("EL QUETZAL", "San Marcos"),
    ("LA REFORMA", "San Marcos"),
    ("PAJAPITA", "San Marcos"),
    ("IXCHIGUÁN", "San Marcos"),
    ("SIPACAPA", "San Marcos"),
    ("ESQUIPULAS PALO GORDO", "San Marcos"),
    ("RÍO BLANCO", "San Marcos"),
    ("SAN LORENZO", "San Marcos"),
    ("LA BLANCA", "San Marcos"),
    ("NUEVO PROGRESO", "San Marcos"),
    ("AYUTLA", "San Marcos"),
    ("CATARINA", "San Marcos"),
    ("SAN JOSÉ OJETENAM", "San Marcos"),
    ("SAN CRISTÓBAL CUCHO", "San Marcos"),
    ("HUEHUETENANGO", "Huehuetenango"),
    ("MALACATANCITO", "Huehuetenango"),
    ("CUILCO", "Huehuetenango"),
    ("SAN PEDRO NECTA", "Huehuetenango"),
    ("SOLOMA", "Huehuetenango"),
    ("SANTA BÁRBARA", "Huehuetenango"),
    ("LA LIBERTAD", "Huehuetenango"),
    ("SAN MIGUEL ACATÁN", "Huehuetenango"),
    ("SAN RAFAEL LA INDEPENDENCIA", "Huehuetenango"),
    ("TODOS SANTOS CUCHUMATÁN", "Huehuetenango"),
    ("SANTA EULALIA", "Huehuetenango"),
    ("TECTITÁN", "Huehuetenango"),
    ("SAN JUAN IXCOY", "Huehuetenango"),
    ("SAN ANTONIO HUISTA", "Huehuetenango"),
    ("SAN SEBASTIÁN COATÁN", "Huehuetenango"),
    ("AGUACATÁN", "Huehuetenango"),
    ("SAN GASPAR IXCHIL", "Huehuetenango"),
    ("SAN JUAN ATITÁN", "Huehuetenango"),
    ("NENTÓN", "Huehuetenango"),
    ("JACALTENANGO", "Huehuetenango"),
    ("IXTAHUCÁN", "Huehuetenango"),
    ("SAN RAFAEL PETZAL", "Huehuetenango"),
    ("PETATAN", "Huehuetenango"),
    ("SAN SEBASTIÁN HUEHUETENANGO", "Huehuetenango"),
    ("SANTIAGO CHIMALTENANGO", "Huehuetenango"),
    ("SANTA ANA HUISTA", "Huehuetenango"),
    ("BARILLAS", "Huehuetenango"),
    ("CONCEPCIÓN HUISTA", "Huehuetenango"),
    ("SANTA CRUZ DEL QUICHÉ", "Quiché"),
    ("ZACUALPA", "Quiché"),
    ("CHAJUL", "Quiché"),
    ("CHICHICASTENANGO", "Quiché"),
    ("PATZITÉ", "Quiché"),
    ("SAN ANTONIO ILOTENANGO", "Quiché"),
    ("SAN PEDRO JOCOPILAS", "Quiché"),
    ("CUNÉN", "Quiché"),
    ("SAN JUAN COTZAL", "Quiché"),
    ("JOYABAJ", "Quiché"),
    ("NEBAJ", "Quiché"),
    ("SAN ANDRÉS SAJCABAJÁ", "Quiché"),
    ("SAN BARTALOMÉ JOCOTENANGO", "Quiché"),
    ("CANILLÁ", "Quiché"),
    ("CHICAMÁN", "Quiché"),
    ("IXCÁN", "Quiché"),
    ("PACHALUM", "Quiché"),
    ("CHICHÉ", "Quiché"),
    ("CHINIQUE", "Quiché"),
    ("USPANTÁN", "Quiché"),
    ("SACAPULAS", "Quiché"),
    ("SALAMÁ", "Baja Verapaz"),
    ("SAN MIGUEL CHICAJ", "Baja Verapaz"),
    ("RABINAL", "Baja Verapaz"),
    ("CUBULCO", "Baja Verapaz"),
    ("GRANADOS", "Baja Verapaz"),
    ("SAN JERÓNIMO", "Baja Verapaz"),
    ("PURULHÁ", "Baja Verapaz"),
    ("EL CHOL", "Baja Verapaz"),
    ("COBÁN", "Alta Verapaz"),
    ("SANTA CRUZ VERAPAZ", "Alta Verapaz"),
    ("SAN CRISTÓBAL VERAPAZ", "Alta Verapaz"),
    ("TACTIC", "Alta Verapaz"),
    ("TAMAHÚ", "Alta Verapaz"),
    ("TUCURÚ", "Alta Verapaz"),
    ("PANZÓS", "Alta Verapaz"),
    ("SENAHÚ", "Alta Verapaz"),
    ("SAN PEDRO CARCHÁ", "Alta Verapaz"),
    ("SAN JUAN CHAMELCO", "Alta Verapaz"),
    ("LANQUÍN", "Alta Verapaz"),
    ("CHISEC", "Alta Verapaz"),
    ("CHAHAL", "Alta Verapaz"),
    ("FRAY BARTOLOMÉ DE LAS CASAS", "Alta Verapaz"),
    ("SAN CATALINA LA TINTA", "Alta Verapaz"),
    ("RAXRUJÀ", "Alta Verapaz"),
    ("CAHABÓN", "Alta Verapaz"),
    ("FLORES", "Petén"),
    ("SAN ANDRÉS", "Petén"),
    ("SAN FRANCISCO", "Petén"),
    ("DOLORES", "Petén"),
    ("SAN LUIS", "Petén"),
    ("SAYAXCHÉ", "Petén"),
    ("MELCHOR DE MENCOS", "Petén"),
    ("POPTÚN", "Petén"),
    ("LAS CRUCES", "Petén"),
    ("EL CHAL", "Petén"),
    ("SAN BENITO", "Petén"),
    ("SANTA ANA", "Petén"),
    ("GUASTATOYA", "El Progreso"),
    ("MORAZÁN", "El Progreso"),
    ("SAN AGUSTÍN ACASAGUASTLAN", "El Progreso"),
    ("SAN CRISTÓBAL ACASAGUASTLAN", "El Progreso"),
    ("EL JÍCARO", "El Progreso"),
    ("SANSARE", "El Progreso"),
    ("SANARATE", "El Progreso"),
    ("SAN ANTONIO LA PAZ", "El Progreso"),
    ("ANTIGUA GUATEMALA", "Sacatepéquez"),
    ("JOCOTENANGO", "Sacatepéquez"),
    ("PASTORES", "Sacatepéquez"),
    ("SUMPANGO", "Sacatepéquez"),
    ("SANTO DOMINGO XENACOJ", "Sacatepéquez"),
    ("SANTIAGO SACATEPÉQUEZ", "Sacatepéquez"),
    ("SAN BARTOLOMÉ MILPAS ALTAS", "Sacatepéquez"),
    ("SAN LUCAS SACATEPÉQUEZ", "Sacatepéquez"),
    ("SANTA LUCÍA MILPAS ALTAS", "Sacatepéquez"),
    ("MAGDALENA MILPAS ALTAS", "Sacatepéquez"),
    ("SANTA MARÍA DE JESÚS", "Sacatepéquez"),
    ("CIUDAD VIEJA", "Sacatepéquez"),
    ("SAN MIGUEL DUEÑAS", "Sacatepéquez"),
    ("ALOTENANGO", "Sacatepéquez"),
    ("SANTA CATARINA BARAHONA", "Sacatepéquez"),
    ("MAZATENANGO", "Suchitepéquez"),
    ("CUYOTENANGO", "Suchitepéquez"),
    ("SAN FRANCISCO ZAPOTITLÁN", "Suchitepéquez"),
    ("SAN BERNARDINO", "Suchitepéquez"),
    ("SAN JOSÉ EL ÍDOLO", "Suchitepéquez"),
    ("SANTO DOMINGO SUCHITEPÉQUEZ", "Suchitepéquez"),
    ("SAMAYAC", "Suchitepéquez"),
    ("SAN PABLO JOCOPILAS", "Suchitepéquez"),
    ("SAN ANTONIO SUCHITEPÉQUEZ", "Suchitepéquez"),
    ("SAN GABRIEL", "Suchitepéquez"),
    ("CHICACAO", "Suchitepéquez"),
    ("SAN JUAN BAUTISTA", "Suchitepéquez"),
    ("SANTO TOMÁS LA UNIÓN", "Suchitepéquez"),
    ("ZUNILITO", "Suchitepéquez"),
    ("PUEBLO NUEVO", "Suchitepéquez"),
    ("RÍO BRAVO", "Suchitepéquez"),
    ("SAN JOSE LA MÁQUINA", "Suchitepéquez"),
    ("PUERTO BARRIOS", "Izabal"),
    ("LIVINGSTON", "Izabal"),
    ("EL ESTOR", "Izabal"),
    ("MORALES", "Izabal"),
    ("ZACAPA", "Zacapa"),
    ("RÍO HONDO", "Zacapa"),
    ("GUALÁN", "Zacapa"),
    ("TECULUTÁN", "Zacapa"),
    ("SAN DIEGO", "Zacapa"),
    ("HUITÉ", "Zacapa"),
    ("CHIQUIMULA", "Chiquimula"),
    ("SAN JUAN ERMITA", "Chiquimula"),
    ("ESQUIPULAS", "Chiquimula"),
    ("CONCEPCIÓN LAS MINAS", "Chiquimula"),
    ("QUETZALTEPEQUE", "Chiquimula"),
    ("SAN JACINTO", "Chiquimula"),
    ("IPALA", "Chiquimula"),
    ("JALAPA", "Jalapa"),
    ("SAN PEDRO PINULA", "Jalapa"),
    ("SAN MANUEL CHAPARRÓN", "Jalapa"),
    ("MONJAS", "Jalapa"),
    ("MATAQUESCUINTLA", "Jalapa"),
    ("JUTIAPA", "Jutiapa"),
    ("EL PROGRESO", "Jutiapa"),
    ("SANTA CATARINA MITA", "Jutiapa"),
    ("AGUA BLANCA", "Jutiapa"),
    ("ASUNCIÓN MITA", "Jutiapa"),
    ("YUPILTEPEQUE", "Jutiapa"),
    ("ATESCATEMPA", "Jutiapa"),
    ("JEREZ", "Jutiapa"),
    ("ZAPOTITLÁN", "Jutiapa"),
    ("JALPATAGUA", "Jutiapa"),
    ("PASACO", "Jutiapa"),
    ("QUEZADA", "Jutiapa"),
    ("SAN ANTONIO AGUAS CALIENTES", "Sacatepéquez"),
    ("SAN MIGUEL PANÁN", "Jutiapa"),
    ("LOS AMATES", "Izabal"),
    ("USUMATLÁN", "Zacapa"),
    ("LA UNIÓN", "Zacapa"),
    ("OLOPA", "Chiquimula"),
    ("EL ADELANTO", "Jutiapa"),
    ("COMAPA", "Jutiapa"),
    ("SAN JOSÉ ACATEMPA", "Jutiapa"),
    ("PATULUL", "Suchitepéquez"),
    ("CABAÑAS", "Zacapa"),
    ("SAN JOSÉ LA ARADA", "Chiquimula"),
    ("SAN LUIS JILITEPEQUE", "Jalapa"),
    ("MOYUTA", "Jutiapa"),
    ("CONGUACO", "Jutiapa"),
    ("ESTANZUELA", "Zacapa"),
    ("SAN JORGE", "Zacapa"),
    ("JOCOTÁN", "Chiquimula"),
    ("CAMOTÁN", "Chiquimula"),
    ("LA DEMOCRACIA", "Huehuetenango"),
    ("CHIANTLA", "Huehuetenango"),
    ("SAN MATEO IXTATÁN", "Huehuetenango"),
    ("ZUNIL", "Quetzaltenango"),
    ("La Tinta", "Alta Verapaz"),
    ("Fray Bartolomé de las Casas", "Alta Verapaz"),
    ("Guanazapa", "Escuintla"),
    ("San Joseé La Máquina", "Suchitepéquez"),
    ("San Cristobal Acasaguastlán", "El Progreso"),
    ("Santa Lucia Utatlán", "Sololá"),
    ("San Antonio la Paz", "El Progreso"),
    ("San Francisco la Unión", "Quiché"),
    ("San Francisco el Alto", "Totonicapán"),
    ("Santiago Sactepéquez", "Sacatepéquez"),
    ("Santa Catalina Ixtahuacán", "Sololá"),
    ("Santo Tomás la Unión", "Suchitepéquez"),
    ("San Andres Semetabaj", "Sololá"),
    ("Melchor", "Petén"),
    ("San Rafael la Independencia", "Huehuetenango"),
    ("El Jícaro", "El Progreso"),
    ("Santiago Chimaltenanango", "Huehuetenango"),
    ("Génova", "Quetzaltenango"),
    ("San Luis", "Petén"),
    ("Sanarate", "El Progreso"),
    ("Pachalum", "Quiché"),
    ("Santa Apolonia", "Chimaltenango"),
    ("San José Ojetenam", "San Marcos"),
    ("Zacapa", "Zacapa"),
    ("San Cristóbal Cucho", "San Marcos"),
    ("Jalpatagua", "Jutiapa"),
    ("San Lucas Sacatepéquez", "Sacatepéquez"),
    ("Nuevo Progreso", "San Marcos"),
    ("San Bartolomé Milpas Altas", "Sacatepéquez"),
    ("Petatán", "Huehuetenango"),
    ("El Progreso", "Jutiapa"),
    ("LA DEMOCARACIA", "Escuintla"),
    ("Pueblo Nuevo", "Suchitepequez"),
    ("RAXUJÀ", "Alta Verapaz"),
    ("HUEHUETENAGO", "Huehuetenango"),
    ("Mazatenango", "Suchitepéquez"),
    ("Patulul", "Suchitepéquez"),
    ("San Bernardino", "Suchitepéquez"),
    ("San Miguel Panán", "Suchitepéquez"),
]

In [17]:
municipios = [
    "Guatemala",
"Santa Catarina Pinula",
"San José Pinula",
"San José del Golfo",
"Palencia",
"Chinautla",
"San Pedro Ayampuc",
"Mixco",
"San Pedro Sacatepéquez",
"San Juan Sacatepéquez",
"San Raymundo",
"Chuarrancho",
"Fraijanes",
"Amatitlán",
"Villa Nueva",
"Villa Canales",
"Petapa",
"Guastatoya",
"Morazán",
"San Agustín Acasaguastlán",
"San Cristóbal Acasaguastlán",
"El Jícaro",
"Sansare",
"Sanarate",
"San Antonio La Paz",
"Antigua Guatemala",
"Jocotenango",
"Pastores",
"Sumpango",
"Santo Domingo Xenacoj",
"Santiago Sacatepéquez",
"San Bartolomé Milpas Altas",
"San Lucas Sacatepéquez",
"Santa Lucía Milpas Altas",
"Magdalena Milpas Altas",
"Santa María de Jesús",
"Ciudad Vieja",
"San Miguel Dueñas",
"Alotenango",
"San Antonio Aguas Calientes",
"Santa Catarina Barahona",
"Chimaltenango",
"San José Poaquil",
"San Martín Jilotepeque",
"Comalapa",
"Santa Apolonia",
"Tecpán Guatemala",
"Patzún",
"Pochuta",
"Patzicía",
"Santa Cruz Balanyá",
"Acatenango",
"Yepocapa",
"San Andrés Itzapa",
"Parramos",
"Zaragoza",
"El Tejar",
"Escuintla",
"Santa Lucía Cotzumalguapa",
"La Democracia",
"Siquinalá",
"Masagua",
"Tiquisate",
"La Gomera",
"Guanagazapa",
"San José",
"Iztapa",
"Palín",
"San Vicente Pacaya",
"Nueva Concepción",
"Sipacate",
"Cuilapa",
"Barberena",
"Santa Rosa de Lima",
"Casillas",
"San Rafael Las Flores",
"Oratorio",
"San Juan Tecuaco",
"Chiquimulilla",
"Taxisco",
"Santa María Ixhuatán",
"Guazacapán",
"Santa Cruz Naranjo",
"Pueblo Nuevo Viñas",
"Nueva Santa Rosa",
"Sololá",
"San José Chacayá",
"Santa María Visitación",
"Santa Lucía Utatlán",
"Nahualá",
"Santa Catarina Ixtahuacán",
"Santa Clara La Laguna",
"Concepción",
"San Andrés Semetabaj",
"Panajachel",
"Santa Catarina Palopó",
"San Antonio Palopó",
"San Lucas Toliman",
"Santa Cruz La Laguna",
"San Pablo La Laguna",
"San Marcos La Laguna",
"San Juan La Laguna",
"San Pedro La Laguna",
"Santiago Atitlán",
"Totonicapán",
"San Cristóbal Totonicapán",
"San Francisco El Alto",
"San Andrés Xecul",
"Momostenango",
"Santa María Chiquimula",
"Santa Lucía La Reforma",
"San Bartolo",
"Quetzaltenango",
"Salcajá",
"Olintepeque",
"San Carlos Sija",
"Sibilia",
"Cabricán",
"Cajolá",
"San Miguel Siguilá",
"Ostuncalco",
"San Mateo",
"Concepción Chiquirichapa",
"San Martín Sacatepéquez",
"Almolonga",
"Cantel",
"Huitán",
"Zunil",
"Colomba",
"San Francisco La Unión",
"El Palmar",
"Coatepeque",
"Génova",
"Flores Costa Cuca",
"La Esperanza",
"Palestina de Los Altos","Mazatenango",
"Cuyotenango",
"San Francisco Zapotitlán",
"San Bernardino",
"San José El Ídolo",
"Santo Domingo Suchitepéquez",
"San Lorenzo",
"Samayac",
"San Pablo Jocopilas",
"San Antonio Suchitepéquez",
"San Miguel Panán",
"San Gabriel",
"Chicacao",
"Patulul",
"Santa Bárbara",
"San Juan Bautista",
"Santo Tomás La Unión",
"Zunilito",
"Pueblo Nuevo ",
"Rio Bravo",
"San José La Máquina",
"Retalhuleu",
"San Sebastián",
"Santa Cruz Muluá",
"San Martín Zapotitlán",
"San Felipe",
"San Andrés Villa Seca",
"Champerico",
"Nuevo San Carlos",
"El Asintal",
"San Marcos",
"San Pedro Sacatepéquez",
"San Antonio Sacatepéquez",
"Comitancillo",
"San Miguel Ixtahuacán",
"Concepción Tutuapa",
"Tacaná",
"Sibinal",
"Tajumulco",
"Tejutla",
"San Rafael Pie de La Cuesta",
"Nuevo Progreso",
"El Tumbador",
"El Rodeo",
"Malacatán",
"Catarina",
"Ayutla",
"Ocós",
"San Pablo",
"El Quetzal",
"La Reforma",
"Pajapita",
"Ixchiguán",
"San José Ojetenam",
"San Cristóbal Cucho",
"Sipacapa",
"Esquipulas Palo Gordo",
"Rio Blanco",
"San Lorenzo",
"La Blanca",
"Huehuetenango",
"Chiantla",
"Malacatancito",
"Cuilco",
"Nentón",
"San Pedro Necta",
"Jacaltenango",
"Soloma",
"Ixtahuacán",
"Santa Bárbara",
"La Libertad",
"La Democracia",
"San Miguel Acatán",
"San Rafael La Independencia",
"Todos Santos Cuchumatán",
"San Juan Atitán",
"Santa Eulalia",
"San Mateo Ixtatán",
"Colotenango",
"San Sebastián Huehuetenango",
"Tectitán",
"Concepción Huista",
"San Juan Ixcoy",
"San Antonio Huista",
"San Sebastián Coatán",
"Barillas",
"Aguacatán",
"San Rafael Petzal",
"San Gaspar Ixchil",
"Santiago Chimaltenango",
"Santa Ana Huista",
"Unión Cantinil",
"Petatán",
"Santa Cruz del Quiché",
"Chiché",
"Chinique",
"Zacualpa",
"Chajul",
"Chichicastenango",
"Patzité",
"San Antonio Ilotenango",
"San Pedro Jocopilas",
"Cunén",
"San Juan Cotzal",
"Joyabaj",
"Nebaj",
"San Andrés Sajcabajá",
"Uspantán",
"Sacapulas",
"San Bartolomé Jocotenango",
"Canillá",
"Chicamán",
"Ixcán",
"Pachalum",
"Salamá",
"San Miguel Chicaj",
"Rabinal",
"Cubulco",
"Granados",
"El Chol",
"San Jerónimo",
"Purulhá",
"Cobán",
"Santa Cruz Verapaz",
"San Cristóbal Verapaz",
"Tactic",
"Tamahú",
"Tucurú",
"Panzós",
"Senahú",
"San Pedro Carchá",
"San Juan Chamelco",
"Lanquín",
"Cahabón",
"Chisec",
"Chahal",
"Fray Bartolomé de Las Casas",
"Santa Catalina la Tinta",
"Raxruhá",
"Flores",
"San José",
"San Benito",
"San Andrés",
"La Libertad",
"San Francisco",
"Santa Ana",
"Dolores",
"San Luis",
"Sayaxché",
"Melchor de Mencos",
"Poptún",
"Las Cruces",
"El Chal",
"Puerto Barrios",
"Livingston",
"El Estor",
"Morales",
"Los Amates",
"Zacapa",
"Estanzuela",
"Río Hondo",
"Gualán",
"Teculután",
"Usumatlán",
"Cabañas",
"San Diego",
"La Unión",
"Huité",
"San Jorge",
"Chiquimula",
"San José La Arada",
"San Juan Erminta",
"Jocotán",
"Camotán",
"Olopa",
"Esquipulas",
"Concepción Las Minas",
"Quetzaltepeque",
"San Jacinto",
"Ipala",
"Jalapa",
"San Pedro Pinula",
"San Luis Jilotepeque",
"San Manuel Chaparrón",
"San Carlos Alzatate",
"Monjas",
"Mataquescuintla",
"Jutiapa",
"El Progreso",
"Santa Catarina Mita",
"Agua Blanca",
"Asunción Mita",
"Yupiltepeque",
"Atescatempa",
"Jerez",
"El Adelanto",
"Zapotitlán",
"Comapa",
"Jalpatagua",
"Conguaco",
"Moyuta",
"Pasaco",
"San José Acatempa",
"Quesada",
].extend(pd.DataFrame(municipios_departamentos)[0].to_list())

In [18]:
df_final.shape

(409, 53)

In [19]:
print(set(df_final['depto'].unique())-set(df_final['depto_mp'].unique())
,set(df_final['depto'].unique())-set(df_final['depto_oj'].unique())
,set(df_final['depto'].unique())-set(df_final['depto_pnc'].unique()))

set() set() set()


In [20]:
mask = (
    (df_final['depto']==df_final['depto_mp'])
    & (df_final['depto']==df_final['depto_pnc'])
    & (df_final['depto']==df_final['depto_oj'])
)
df_final = df_final[mask] # Solo para extraer todos los datos de ese lugar

In [21]:
print(df_final['depto'].unique())
print(df_final['Ubicación '].unique())

['Chimaltenango' 'Jutiapa' 'Huehuetenango' 'Quetzaltenango' 'Sacatepéquez'
 'Guatemala' 'San Marcos' 'Santa Rosa' 'Zacapa' 'Alta Verapaz'
 'Chiquimula' 'Quiché' 'Retalhuleu' 'Suchitepéquez' 'Sololá'
 'Baja Verapaz' 'Petén' 'Izabal' 'El Progreso' 'Escuintla' 'Jalapa'
 'Totonicapán']
['Acatenango' 'Agua Blanca' 'Aguacatán' 'Almolonga' 'Alotenango'
 'Amatitlán' 'Antigua Guatemala' 'Asunción Mita' 'Atescatempa' 'Ayutla'
 'Barberena' 'Cabañas' 'Cabricán' 'Cahabón' 'Cajolá' 'Camotán' 'Canillá'
 'Cantel' 'Casillas' 'Catarina' 'Chahal' 'Chajul' 'Champerico' 'Chiantla'
 'Chicacao' 'Chicamán' 'Chichicastenango' 'Chiché' 'Chinautla' 'Chinique'
 'Chiquimulilla' 'Chisec' 'Chuarrancho' 'Ciudad Vieja' 'Coatepeque'
 'Cobán' 'Colomba' 'Colotenango' 'Comapa' 'Comitancillo' 'Concepción'
 'Concepción Chiquirichapa' 'Concepción Huista' 'Concepción Las Minas'
 'Concepción Tutuapa' 'Conguaco' 'Cubulco' 'Cuilapa' 'Cuilco' 'Cunén'
 'Cuyotenango' 'El Adelanto' 'El Asintal' 'El Chal ' 'El Estor'
 'El Jícaro' 'El

### Normalizar los departamentos y municipios (ubicaciones)

In [22]:
# Normalización de municipios y creación de lista de tuplas (municipio, departamento)
import unicodedata
def normalize_text(text):
    text = str(text).strip().lower()
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    return text
df_final["MUNICIPIO_NORM"] = df_final['Ubicación '].apply(normalize_text)
df_final["DEPARTAMENTO_NORM"] = df_final["depto"].apply(normalize_text)
municipios_departamentos = list(zip(df_final["MUNICIPIO_NORM"], df_final["DEPARTAMENTO_NORM"]))
municipios_departamentos[:10]  # Muestra los primeros 10 para verificar

[('acatenango', 'chimaltenango'),
 ('agua blanca', 'jutiapa'),
 ('aguacatan', 'huehuetenango'),
 ('almolonga', 'quetzaltenango'),
 ('alotenango', 'sacatepequez'),
 ('amatitlan', 'guatemala'),
 ('antigua guatemala', 'sacatepequez'),
 ('asuncion mita', 'jutiapa'),
 ('atescatempa', 'jutiapa'),
 ('ayutla', 'san marcos')]

In [23]:
df_final['DEPARTAMENTO_NORM']

0       chimaltenango
1             jutiapa
2       huehuetenango
3      quetzaltenango
4        sacatepequez
            ...      
404            quiche
405       el progreso
406     chimaltenango
407    quetzaltenango
408     suchitepequez
Name: DEPARTAMENTO_NORM, Length: 333, dtype: object

In [24]:
# Supón que tienes una lista oficial de tuplas (municipio, departamento) llamada municipios_departamentos_oficial
# Ejemplo: municipios_departamentos_oficial = [("guatemala", "guatemala"), ("mixco", "guatemala"), ...]
from difflib import get_close_matches
def match_municipio(muni, muni_list):
    match = get_close_matches(muni, muni_list, n=1, cutoff=0.6)
    return match[0] if match else None

# Extrae la lista de municipios oficiales normalizados
municipios_oficiales = [normalize_text(m) for m, d in municipios_departamentos]
df_final["MUNICIPIO_NORM_MATCH"] = df_final["MUNICIPIO_NORM"].apply(lambda x: match_municipio(x, municipios_oficiales))

In [25]:
df_final["MUNICIPIO_NORM_MATCH"] = df_final['Ubicación '].apply(lambda x: match_municipio(x, municipios_oficiales))

In [26]:
df_final[df_final["MUNICIPIO_NORM_MATCH"].isnull()]['Ubicación '].shape

(1,)

In [27]:
df_final[df_final["MUNICIPIO_NORM_MATCH"].isnull()]['MUNICIPIO_NORM'].unique()

array(['ocos'], dtype=object)

In [28]:
df_final["MUNICIPIO_NORM_MATCH"].value_counts().sort_values(ascending=False)

MUNICIPIO_NORM_MATCH
san jorge                 16
la libertad                2
san pedro sacatepequez     2
petatan                    2
santa barbara              2
                          ..
canilla                    1
cantel                     1
casillas                   1
catarina                   1
barberena                  1
Name: count, Length: 311, dtype: int64

In [29]:
df_final["DEPARTAMENTO_NORM"].value_counts().sort_values(ascending=False)

DEPARTAMENTO_NORM
huehuetenango     33
san marcos        29
zacapa            25
quetzaltenango    23
quiche            21
suchitepequez     21
solola            18
alta verapaz      17
el progreso       17
sacatepequez      16
guatemala         16
chimaltenango     15
santa rosa        14
escuintla         13
chiquimula        10
retalhuleu         8
baja verapaz       8
totonicapan        7
jutiapa            6
jalapa             6
izabal             5
peten              5
Name: count, dtype: int64

In [30]:
print(len([col.split('_dept')[0] for col in df_final.columns if not re.search(r'_dept', col)]),
      len([col for col in df_final.columns if re.search(r'_dept', col)]))

32 24


In [31]:
[col.split('_dept')[0].strip() for col in df_final.columns if not re.search(r'_dept', col)]

['Ubicación',
 'Salas (mixtas)',
 'Tribunales de Sentencia',
 'Juzgados de Primera Instancia Penal',
 'Juzgados de Primera Instancia Penal, Narcoactividad y Delitos contral el Ambiente',
 'Juzgados de Adolescentes y Niñez',
 'Juzgados de Ejecución',
 'Juzgados de Paz Penal y Comunitarios',
 'Juzgados de Violencia Contra la Mujer y Femicidio',
 'Tribunales de sentencia Violencia Contra la Mujer y Femicidio',
 'Juzgados y Tribunales de Mayor Riesgo',
 'Total',
 'depto_oj',
 'Fiscalías Regionales',
 'Fiscalías Distritales',
 'Fiscalías Municipales',
 'Agencias Fiscales',
 'depto_mp',
 'Coordinadora departamental',
 'Coordinadora municipal',
 'Secciones especiales',
 'TOTAL CORDINACIONES',
 'depto',
 'Comisarias (oficina de atención a la víctima)',
 'Estaciones',
 'Subestaciones',
 'División de investigación criminal (DEIC)',
 'Totales',
 'depto_pnc',
 'MUNICIPIO_NORM',
 'DEPARTAMENTO_NORM',
 'MUNICIPIO_NORM_MATCH']

In [32]:
set([col.split('_dept')[0].strip() for col in df_final.columns if not re.search(r'_dept', col)])-set([col.split('_dept')[0].strip()for col in df_final.columns if re.search(r'_dept', col)])

{'DEPARTAMENTO_NORM',
 'MUNICIPIO_NORM',
 'MUNICIPIO_NORM_MATCH',
 'Ubicación',
 'depto',
 'depto_mp',
 'depto_oj',
 'depto_pnc'}

In [33]:
final_cols = [col.strip()for col in df_final.columns if re.search(r'_dept', col)]
final_cols.extend(['DEPARTAMENTO_NORM',
       'MUNICIPIO_NORM_MATCH'])
final_cols

['Salas (mixtas)_dept',
 'Tribunales de Sentencia_dept',
 'Juzgados de Primera Instancia Penal_dept',
 'Juzgados de Primera Instancia Penal, Narcoactividad y Delitos contral el Ambiente_dept',
 'Juzgados de Adolescentes y Niñez_dept',
 'Juzgados de Ejecución_dept',
 'Juzgados de Paz Penal y Comunitarios_dept',
 'Juzgados de Violencia Contra la Mujer y Femicidio_dept',
 'Tribunales de sentencia Violencia Contra la Mujer y Femicidio_dept',
 'Juzgados y Tribunales de Mayor Riesgo_dept',
 'Total_dept',
 'Fiscalías Regionales_dept',
 'Fiscalías Distritales_dept',
 'Fiscalías Municipales_dept',
 'Agencias Fiscales _dept',
 'Coordinadora departamental_dept',
 'Coordinadora municipal_dept',
 'Secciones especiales_dept',
 'TOTAL CORDINACIONES _dept',
 'Comisarias (oficina de atención a la víctima) _dept',
 'Estaciones _dept',
 'Subestaciones_dept',
 'División de investigación criminal (DEIC)_dept',
 'Totales_dept',
 'DEPARTAMENTO_NORM',
 'MUNICIPIO_NORM_MATCH']

In [34]:
df_final = df_final[final_cols].fillna(0)
df_final = df_final.rename(columns={
    col: col.replace('_dept', '') for col in df_final.columns if col.endswith('_dept')
})

In [ ]:
df_final.to_csv(
        f"./dataset/variables_alternas_procesadas/distribucion_de_servicios_de_justicia.csv", 
        index=False
    )

In [36]:
df_final

,Salas (mixtas),Tribunales de Sentencia,Juzgados de Primera Instancia Penal,"Juzgados de Primera Instancia Penal, Narcoactividad y Delitos contral el Ambiente",Juzgados de Adolescentes y Niñez,Juzgados de Ejecución,Juzgados de Paz Penal y Comunitarios,Juzgados de Violencia Contra la Mujer y Femicidio,Tribunales de sentencia Violencia Contra la Mujer y Femicidio,Juzgados y Tribunales de Mayor Riesgo,...,Coordinadora municipal,Secciones especiales,TOTAL CORDINACIONES,Comisarias (oficina de atención a la víctima),Estaciones,Subestaciones,División de investigación criminal (DEIC),Totales,DEPARTAMENTO_NORM,MUNICIPIO_NORM_MATCH
0,0,2,0,1,1,0,0,1,1,0,...,0.0,0.0,1,1,4,15,1,21,chimaltenango,acatenango
1,0,3,0,2,1,0,0,1,1,0,...,0.0,0.0,1,1,6,22,1,30,jutiapa,agua blanca
2,0,2,0,1,1,0,1,1,1,0,...,0.0,0.0,6,1,7,35,1,44,huehuetenango,aguacatan
3,0,6,1,8,3,2,1,1,1,2,...,0.0,0.0,5,1,7,28,1,37,quetzaltenango,almolonga
4,1,1,1,1,1,0,1,0,0,0,...,0.0,0.0,1,1,3,17,1,22,sacatepequez,alotenango
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,1,2,0,3,1,0,0,1,1,0,...,0.0,0.0,3,1,8,23,1,33,quiche,zacualpa
405,0,1,0,1,0,0,0,0,0,0,...,0.0,0.0,1,1,3,11,1,16,el progreso,zapotitlan
406,0,2,0,1,1,0,0,1,1,0,...,0.0,0.0,1,1,4,15,1,21,chimaltenango,zaragoza
407,0,6,1,8,3,2,1,1,1,2,...,0.0,0.0,5,1,7,28,1,37,quetzaltenango,zunil
